In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MeshCat as mc
import Distributions
import Random
import Combinatorics

using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using CSV
using DataFrames
using Distributions
using Combinatorics

  Activating project at `c:\Users\Daniel\Documents\Python-Projects\AiPEX-Projects\warmstarting_NLPs\JL_presolves`
┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing C:\Users\Daniel\Documents\Python-Projects\AiPEX-Projects\warmstarting_NLPs\JL_presolves\Manifest.toml:0


In [2]:
include(joinpath(@__DIR__, "piano_dynamics.jl"))


  Activating project at `c:\Users\Daniel\Documents\Python-Projects\AiPEX-Projects\warmstarting_NLPs\JL_presolves`
┌ Warning: The active manifest file at `C:\Users\Daniel\Documents\Python-Projects\AiPEX-Projects\warmstarting_NLPs\JL_presolves\Manifest.toml` has an old format that is being maintained.
│ To update to the new format, which is supported by Julia versions ≥ 1.6.2, run `import Pkg; Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
│ To then record the julia version re-resolve with `Pkg.resolve()` and if there are resolve conflicts consider `Pkg.update()`.
└ @ Pkg.Types C:\Users\Daniel\.julia\juliaup\julia-1.10.2+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Pkg\src\manifest.jl:318
  No Changes to `C:\Users\Daniel\Documents\Python-Projects\AiPEX-Projects\warmstarting_NLPs\JL_presolves\Project.toml`
  No Changes to `C:\Users\Daniel\Documents\Python-Projects\AiPEX-Projects\warmstarting_NLPs\JL_presolves\Manifest.toml`


solve_hand_dircol (generic function with 2 methods)

### Sample parameters 

In [3]:
function generate_piano_params(N_param_samples)::Matrix{Float64}
    # Choose 3 notes to play from an octave range: 12_P_3
    notes = 1:8
    # notes_distribution = DiscreteUniform(1,12)
    notes_distribution = permutations(notes, 3)
    notes = rand(collect(notes_distribution), N_param_samples)
    notes = stack(notes)
    notes = transpose(notes)
    # @show size(notes)
    # @show typeof(notes)
    # @show notes
    # println()


    # Choose note durations
    t_end = 3.0
    note_timings_distribution = Product([Uniform(0, t_end), Uniform(0, t_end)])
    notes_timings = rand(note_timings_distribution, N_param_samples)
    sort!(notes_timings,dims=1)
    notes_timings = transpose(notes_timings)
    # @show size(notes_timings)
    # @show typeof(notes_timings)
    # @show notes_timings
    # println()

    # Choose fingers to play 3 notes: 5_P_3
    finger_idx = 1:5
    # finger_distribution = DiscreteUniform(1,5)
    fingers_distribution = permutations(finger_idx, 3)
    fingers = rand(collect(fingers_distribution), N_param_samples)
    fingers = stack(fingers)
    fingers = transpose(fingers)
    # @show size(fingers)
    # @show typeof(fingers)
    # @show fingers

    # Joint parameter distribution sample
    θ_samples = hcat(notes, notes_timings, fingers)

    return θ_samples
end



generate_piano_params (generic function with 1 method)

In [4]:
df = DataFrame(params = Vector{Vector{Float64}}(), X=Vector{Vector{Vector{Float64}}}(), U=Vector{Vector{Vector{Float64}}}(), obj = Float64[], solve_time_sec = Float64[], term_status = MOI.TerminationStatusCode[])

Random.seed!(123)

N_param_samples = 20 # Number of parameters to sample for the OCP
nlp_params = generate_piano_params(N_param_samples)
# θ_samples = eachrow(θ_samples)
# @show θ_samples


for θ in eachrow(nlp_params)
    println(θ)
end

[5.0, 2.0, 3.0, 0.8297923385248999, 2.9503071333595194, 3.0, 1.0, 5.0]
[5.0, 6.0, 8.0, 0.7020504173221651, 2.6426924724474192, 1.0, 5.0, 4.0]
[8.0, 1.0, 7.0, 0.3958311976184986, 1.142848137858326, 3.0, 5.0, 4.0]
[2.0, 5.0, 7.0, 0.2648730573968353, 0.9405147435231863, 3.0, 5.0, 1.0]
[5.0, 2.0, 4.0, 1.390829232974743, 2.1409077674586237, 3.0, 2.0, 1.0]
[4.0, 1.0, 8.0, 0.27165349265334193, 0.6177747284601298, 5.0, 1.0, 3.0]
[1.0, 4.0, 6.0, 0.9343425021151587, 1.7457370271629369, 3.0, 5.0, 2.0]
[8.0, 4.0, 2.0, 0.3634425615543808, 0.6135894519610784, 5.0, 1.0, 2.0]
[5.0, 6.0, 4.0, 0.05571599876981548, 1.160070488726861, 3.0, 4.0, 5.0]
[3.0, 6.0, 2.0, 0.21654217110422047, 2.7427397578313797, 4.0, 5.0, 2.0]
[5.0, 2.0, 6.0, 1.1621221867606972, 2.3157490675343317, 1.0, 3.0, 4.0]
[7.0, 5.0, 6.0, 1.132149778298261, 2.626664998678856, 5.0, 3.0, 2.0]
[1.0, 4.0, 3.0, 0.793704421085792, 2.705631973591245, 5.0, 1.0, 4.0]
[4.0, 7.0, 1.0, 2.19671445365391, 2.5940792760935096, 4.0, 5.0, 3.0]
[3.0, 8.0, 2

In [5]:

# X, U, t_vec, params, objs = solve_hand_dircol(nlp_params[1,:]; verbose=true)
X, U, obj, solve_time_sec, term_status, t_vec, params = solve_hand_dircol(nlp_params[1,:]; verbose=true)

finger_constraints = FingerConstraint[FingerConstraint(0.0, 1.0, 1, 1)]
Initial state: [0.00997183, -0.209972, 0.00990915, -1.49735e-10, 0.085188, 5.36454e-11, 2.23554e-11, 0.447411, 0.463769, 5.23797e-11, 1.82296e-11, 0.574483, 0.588057, -5.19199e-11, 0.003956, 0.591765, 0.602195, 0.184569, -0.066329, -0.060192, 0.662445, 0.672905, 0.36645, 0.69654, -0.2094, 0.621309, 0.187236, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
typeof(robohand_end_effector_cost) = typeof(robohand_end_effector_cost)
---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :finite (FiniteDiff.jl)---
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

***************************************

MethodError: MethodError: Cannot `convert` an object of type 
  Tuple{Vector{Float64},Float64,Float64,MathOptInterface.TerminationStatusCode} to an object of type 
  Tuple{Vector,Float64}

Closest candidates are:
  convert(::Type{T}, ::Tuple{Vararg{Any, N}}) where {N, T<:Tuple}
   @ Base essentials.jl:457
  convert(::Type{T}, !Matched::T) where T<:Tuple
   @ Base essentials.jl:456
  convert(::Type{T}, !Matched::T) where T
   @ Base Base.jl:84
  ...
